# Identifying Emerging and Growing Fields: Evaluating Europe's Role in the Global Innovation Landscape

### Prerrequisites
This notebook has been written for an audience already familiar with TIP. For users completely new to the platform, we recommend to follow the [TIP basic course](https://e-courses.epo.org/course/view.php?id=416) before reading this analysis. 
### Disclaimer 
This notebook is published as a showcase of the capabilities of TIP as a data processing environment. It is not intended to derive any statistical conclusion other than the presentation of the retrieved data. The EPO is not expressing any opinion extrapolated from the analysis.

## Introduction

In the face of increasing global competition, the **European Commission** has acknowledged the importance of *emerging sectors* to drive innovation and secure its economic future. While there are investments being made, these are often too limited or fragmented to keep pace with global leaders such as the **US**, **Japan**, **South Korea**, and **China**. *Emerging sectors* defined as fields characterised by their novelty, rapid development, and transformative potential are essential for addressing challenges like **climate change**, **digital transformation**, and advancements in **healthcare**.

In light of this it is important to carefully assess areas where Member States might be trailing behind other innovation leaders. Identifying emerging fields of technology can be challenging, as these areas are not always immediately apparent. Once they are recognised, it becomes quite essential to evaluate Europe's position in these fields and consider whether the current level of investment is sufficient to sustain or enhance global competitiveness. This preliminary analysis aims to provide initial insights into how TIP (Technology and Innovation Platform) can be utilised to investigate whether Europe is dedicating adequate resources to these emerging areas and where there might be risks of falling behind. By examining patent trends and the geographic distribution of applicants within these technologies, the objective is to identify potential gaps that may require further attention. we aim to uncover areas requiring urgent attention to ensure **Europe’s long-term competitiveness** and leadership in the global innovation landscape, a focus that is highly relevant as highlighted in the report [*The future of European competitiveness: A competitiveness strategy for Europe*](https://commission.europa.eu/document/download/97e481fd-2dc3-412d-be4c-f152a8232961_en?filename=The%20future%20of%20European%20competitiveness%20_%20A%20competitiveness%20strategy%20for%20Europe.pdf).

It should be noted that this analysis is deliberately simplified, serving as a starting point to guide a more detailed exploration of Europe’s positioning within the global innovation landscape. This analysis was conducted in February 2025, and the results discussed reflect the data available at that time. The process can be repeated, but it is important to consider that results may vary over time.

In [1]:
from epo.tipdata.patstat import PatstatClient
from epo.tipdata.patstat.database.models import (
    TLS201_APPLN,
    TLS231_INPADOC_LEGAL_EVENT,
    TLS803_LEGAL_EVENT_CODE,
    TLS224_APPLN_CPC,
    TLS206_PERSON,
    TLS207_PERS_APPLN,
    TLS225_DOCDB_FAM_CPC,
    TLS801_COUNTRY

)


patstat = PatstatClient(env="PROD")

db = patstat.orm()

In [2]:
from sqlalchemy import and_, case, func, select, distinct, or_
import requests
import zipfile
import io
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## CPC System

A critical step in understanding global innovation trends is identifying the technological areas that have experienced the most significant activity in recent years. To achieve this, we analyse *patent classifications* using the **Cooperative Patent Classification (CPC)** system, which provides a detailed taxonomy of technologies. Specifically, we focus on **CPC groups**, a level of classification that aggregates patents based on shared technological characteristics, offering a broader perspective on innovation trends.

For this analysis, we examine patents filed over the past decade, starting from **2013**. This time frame reflects the most recent developments and allows us to capture trends in rapidly evolving and *emerging fields*. By counting the occurrences of **CPC groups** associated with *patent families*, we highlight the most frequently represented technological areas. These counts act as a proxy for *innovation intensity*, illustrating which sectors are experiencing the highest levels of research and development globally.

The results reveal the dominant areas of technological activity, providing insights into where *innovation efforts* might be concentrated. 


In [3]:
cpc_group_count_query = (
    db.query(
        TLS225_DOCDB_FAM_CPC.cpc_class_symbol, 
        func.count(TLS225_DOCDB_FAM_CPC.docdb_family_id).label("cpc_count")  
    )
    .join(
        TLS201_APPLN, TLS225_DOCDB_FAM_CPC.docdb_family_id == TLS201_APPLN.docdb_family_id  
    )
    .filter(
        TLS201_APPLN.earliest_publn_year >= 2013,
        TLS201_APPLN.earliest_publn_year != 9999
    )
    .group_by(TLS225_DOCDB_FAM_CPC.cpc_class_symbol)  
    .order_by(func.count(TLS225_DOCDB_FAM_CPC.docdb_family_id).desc())  
)


cpc_group_count_df = patstat.df(cpc_group_count_query)


cpc_group_count_df


,cpc_class_symbol,cpc_count
0,A61P 35/00,953446
1,Y02E 60/10,890273
2,A61K 45/06,561923
3,A61P 43/00,388999
4,A61K2039/505,379663
...,...,...
241695,F04B 1/0478,1
241696,H01L2224/0546,1
241697,G03B 41/026,1
241698,C10M2215/2275,1


After identifying the most frequently occurring **CPC groups** over the past decade, we proceed by selecting the top **50 CPC groups** for further analysis. This selection, while somewhat arbitrary, is guided by the principle of focusing on the most active and influential *technological areas*. By narrowing the scope to the top **50**, we aim to concentrate on the key drivers of *innovation* without diluting the analysis with less impactful fields. In this case, the choice of the number 50 was made arbitrarily to demonstrate how such an analysis might be conducted, but this limit is flexible and can be adjusted based on the specific needs of the study.


In [4]:
top_cpc_classes = cpc_group_count_df['cpc_class_symbol'].tolist()

top_cpc_classes[:50] 

['A61P  35/00',
 'Y02E  60/10',
 'A61K  45/06',
 'A61P  43/00',
 'A61K2039/505',
 'Y02P  70/50',
 'A61P  29/00',
 'Y02T  10/70',
 'A23V2002/00',
 'G06N   3/08',
 'Y02E  10/50',
 'Y02A  50/30',
 'A61P  25/00',
 'C07D 471/04',
 'H01M  10/0525',
 'Y02D  30/70',
 'Y02P  70/10',
 'C07K2317/92',
 'G06N  20/00',
 'C07K2317/76',
 'Y02D  10/00',
 'G06N   3/045',
 'C07D 487/04',
 'A61K  38/00',
 'A61P  25/28',
 'A61K   9/0019',
 'C07K2317/24',
 'H04W  72/23',
 'C07D 401/14',
 'H04N  19/176',
 'A61P   3/10',
 'Y02T  10/7072',
 'A61P  11/00',
 'A61P   9/00',
 'H04L   5/0053',
 'Y02T  10/12',
 'H01M  10/052',
 'A61P  35/02',
 'Y02E  60/50',
 'F21Y2115/10',
 'Y02E  10/72',
 'C07K2317/565',
 'A61P  31/04',
 'A61P  17/00',
 'A61K  31/519',
 'C07D 401/12',
 'B32B  27/08',
 'A61P   9/10',
 'B32B   7/12',
 'H04W   4/80']

With the list of top **CPC groups** identified, the next step is to analyse the *applicants* contributing to these technological areas. The goal is to determine which *entities*, particularly those from the five influential countries (**US**, **JP**, **CN**, **KR**, and **EP contracting states**), are driving *innovation* in these fields. This step focuses on understanding the *geographic* and *organisational* distribution of patent activity in the identified high-priority technologies.


## IPFs for identifying **high-value inventions**

To ensure the robustness and comparability of our analysis, this study focuses on **International Patent Families (IPFs)**. **IPFs** represent high-value inventions for which patent protection has been sought in more than one jurisdiction. This approach provides a reliable and neutral proxy for inventive activity, allowing us to analyse technological developments with greater accuracy and consistency than would be possible by examining individual patent applications alone.

Patents are inherently territorial, with protection granted only within the jurisdiction of the patent office where the application is filed. Consequently, the count of patent applications at a national or regional level does not necessarily equate to the number of unique inventions. Many patent filings may represent minor refinements rather than distinct innovations. **IPFs** address this limitation by grouping patents filed across jurisdictions that relate to the same invention. Each **IPF** is defined as a unique invention for which patents have been filed in at least two countries, thereby ensuring that only inventions deemed commercially significant by their applicants are included.

This methodology reduces biases that can arise from variations in patent filing practices across different jurisdictions. By aggregating related patents into families, **IPFs** create a homogeneous population of inventions that can be compared across countries and companies. This global perspective provides a more accurate measure of inventive activity, as it reflects innovations with broader market potential and strategic importance.

For this analysis, **IPFs** were identified using two key criteria. The first required that patent families include applications filed in at least two jurisdictions, ensuring that the inventions represented are of international significance. The second focused on families associated with organisations, highlighting commercially oriented inventions that are more likely to influence technological and industrial progress. This definition of an *international patent family* is one of the available options. TIP allows for defining these families in other ways as well. This serves as a starting point, and from here, decisions can be made to include or exclude certain cases based on the specific objectives of the analysis.

By concentrating on **IPFs**, this study captures the most valuable and impactful innovations driving technological progress. This approach not only highlights high-priority fields but also provides a comprehensive understanding of how inventive activity is distributed globally, offering meaningful insights into regional and sectoral dynamics.


In [ ]:
ipf_families_subquery = (
    db.query(TLS201_APPLN.docdb_family_id)
    .filter(
        # Families belonging to organizations
        TLS201_APPLN.docdb_family_id.in_(
            db.query(TLS201_APPLN.docdb_family_id)
            .join(TLS801_COUNTRY, TLS201_APPLN.appln_auth == TLS801_COUNTRY.ctry_code)
            .filter(TLS801_COUNTRY.organisation_flag == 'y')
            .distinct()
        )
        | 
        # Families with applications from 2+ distinct authorities
        TLS201_APPLN.docdb_family_id.in_(
            db.query(TLS201_APPLN.docdb_family_id)
            .filter(TLS201_APPLN.ipr_type != 'UM')  # Exclude utility models
            .group_by(TLS201_APPLN.docdb_family_id)
            .having(func.count(func.distinct(TLS201_APPLN.appln_auth)) >= 2)
        )
    )
    .distinct()
    .subquery()
)




ipf_families_query = (
    db.query(
        TLS201_APPLN.appln_id,               
        TLS201_APPLN.docdb_family_id,        
        TLS225_DOCDB_FAM_CPC.cpc_class_symbol,  
        TLS206_PERSON.person_ctry_code,
        TLS206_PERSON.person_id, 
        TLS206_PERSON.han_name, 
        TLS206_PERSON.psn_name, 
        TLS201_APPLN.earliest_publn_year     
    )
    .join(
        TLS207_PERS_APPLN, TLS201_APPLN.appln_id == TLS207_PERS_APPLN.appln_id 
    )
    .join(
        TLS206_PERSON, TLS207_PERS_APPLN.person_id == TLS206_PERSON.person_id  
    )
    .join(
        TLS225_DOCDB_FAM_CPC, TLS201_APPLN.docdb_family_id == TLS225_DOCDB_FAM_CPC.docdb_family_id  
    )
    .filter(
        TLS201_APPLN.docdb_family_id.in_(ipf_families_subquery),  
        TLS201_APPLN.earliest_publn_year >= 2013,   
        TLS201_APPLN.earliest_publn_year <= 2022,   
        TLS201_APPLN.earliest_publn_year != 9999,   
        TLS225_DOCDB_FAM_CPC.cpc_class_symbol.in_(top_cpc_classes),  
        TLS207_PERS_APPLN.applt_seq_nr != 0,       
        TLS206_PERSON.person_ctry_code.isnot(None),  
        TLS206_PERSON.person_ctry_code.in_([
            "US", "EP", "JP", "CN", "KR", "AL", "AT", "BE", "BG", "HR", "CY", 
            "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IS", "IE", "IT", 
            "LV", "LI", "LT", "LU", "MT", "MC", "NL", "MK", "NO", "PL", "PT", 
            "RO", "SM", "RS", "SK", "SI", "ES", "SE", "CH", "TR", "UK"
        ]),  
    )
    .order_by(TLS201_APPLN.docdb_family_id)  
)


ipf_families_df = patstat.df(ipf_families_query)
ipf_families_df

/tmp/ipykernel_418/3238142015.py:48: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  TLS201_APPLN.docdb_family_id.in_(ipf_families_subquery),


Given the large number of **European Patent Convention (EPC)** contracting countries, we simplify the analysis by grouping all these countries under the unified designation "**EP**". These countries are the current contracting parties, and any changes will be updated accordingly. This approach reduces complexity in geographic aggregation while retaining the ability to compare Europe's overall contribution against other major innovation regions, such as the **US**, **JP**, **CN**, and **KR**.


In [ ]:
epc_country_codes = ["AL", "AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", 
    "GR", "HU", "IS", "IE", "IT", "LV", "LI", "LT", "LU", "MT", "MC", "NL", 
    "MK", "NO", "PL", "PT", "RO", "SM", "RS", "SK", "SI", "ES", "SE", "CH", 
    "TR", "UK"]

ipf_families_df["person_ctry_code"] = ipf_families_df["person_ctry_code"].replace(
    epc_country_codes, "EP"
)

ipf_families_df = ipf_families_df.sort_values(by="docdb_family_id")
ipf_families_df

In the initial stages of this analysis, we focused on specific **CPC** groups to identify detailed technological contributions and their applications. This approach provided a granular view of emerging technologies, allowing us to highlight precise areas of innovation. However, to facilitate a more general interpretation and align with the broader goals of the study, we now shift our focus to **CPC subclasses**, represented by the first four characters of the **CPC** codes.

By adopting this broader lens, we can better understand which macro-areas are leading global innovation efforts. This shift also enables a clearer comparison of technological trends across regions, providing actionable insights for strategic decision-making. The combination of initial detailed analysis and subsequent generalisation may ensure that the study captures both the micro and macro aspects of technological innovation.


In [ ]:
ipf_families_df['cpc_class_prefix'] = ipf_families_df['cpc_class_symbol'].str[:4]

## Adding labels to the Subclasses

To provide a more comprehensive view of the **CPC** subclasses, we can enrich the data with their corresponding names and labels, which are crucial for interpreting the technological fields represented by these subclasses. By downloading the **CPC Title List** from the official **CPC** website, we can match the subclass codes with their descriptive labels, offering a clearer understanding of the technologies behind each classification.

At this stage, we have decided not to focus on the group level, which is very specific to particular technologies, as we aim to maintain a broader perspective. Adding labels from an external website demonstrates what TIP has the potential to do – leveraging external data sources and adding additional layers to those already present in TIP.

By incorporating the subclass names and labels into your analysis, you can provide a clearer, more interpretable view of the technological fields under consideration. This step also ensures that you remain flexible with future changes to the CPC system, as the URL and process are designed to adapt to different editions. This enhanced dataset will allow you to better understand and visualise the contribution of various countries and regions in emerging technological areas.

In [ ]:
prefixes = ipf_families_df['cpc_class_prefix'].unique()
prefixes

In [ ]:
def openCPCFiles(CPCEdition, prefixes):
    """
    Download and process all text files from the specified CPC ZIP folder.
    
    Args:
        CPCEdition (str): The CPC edition year and month (e.g., '202408').
        prefixes (list): A list of CPC class prefixes to filter the lines.
    
    Returns:
        pd.DataFrame: A DataFrame containing rows filtered by the given prefixes.
    """
    # URL of the ZIP file on the CPC website
    url = f"https://www.cooperativepatentclassification.org/sites/default/files/cpc/bulk/CPCTitleList{CPCEdition}.zip" 

    # Download ZIP file
    response = requests.get(url)
    if response.status_code == 200:
        # Open ZIP file from response content
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Initialize a list to hold data
            data = []
            # Iterate over all files in the ZIP
            for file_name in z.namelist():
                # Open and read each text file
                with z.open(file_name) as file:
                    # Read lines from the file
                    lines = file.readlines()
                    # Process all lines (no skipping)
                    for line in lines:
                        decoded_line = line.decode('utf-8').strip()
                        # Check if the line starts with any of the prefixes followed by a tab or '/'
                        if any(decoded_line.startswith(prefix + "\t") or decoded_line.startswith(prefix + "/") for prefix in prefixes):
                            data.append({'FileName': file_name, 'LineContent': decoded_line})

            # Convert the list to a DataFrame
            df = pd.DataFrame(data)
            return df
    else:
        print(f"Failed to download file: {response.status_code}")
        return None

cpc_edition = "202501"
cpc_data = openCPCFiles(cpc_edition, prefixes)

if cpc_data is not None:
    print(cpc_data)

In [ ]:
def processLineContent(df):
    df[['Prefix', 'Label']] = df['LineContent'].str.extract(r'([A-Z0-9/]+)\s+(.*)', expand=True)
    df['Prefix'] = df['Prefix'].apply(lambda x: x[:4])
    df['Label'] = df['Label'].str.replace(r'[^\x00-\x7F]+', '', regex=True)  

    return df
    
if cpc_data is not None:
    cpc_data = processLineContent(cpc_data)
    print(cpc_data)

In [ ]:
cpc_data.rename(columns={'Prefix': 'cpc_class_prefix'}, inplace=True)

merged_data = pd.merge(ipf_families_df, cpc_data[['cpc_class_prefix', 'Label']], 
                       on='cpc_class_prefix', 
                       how='left')

merged_data

## WIPO - IPC Concordance Table to identify the fields

As mentioned, we want to maintain a broader perspective and focus on larger fields that are innovatively significant, without delving too deeply into the specifics of individual technologies. Dividing the subclasses into broader fields can be done by using external sources such as the WIPO-IPC concordance table. This approach allows us to group related subclasses into wider technological areas, providing a more general overview while still capturing key trends and developments in innovation.

The **WIPO IPC Technology Concordance Table** offers a valuable framework for macro-level analysis of technological trends by grouping **IPC** subclasses into broader technology macrofields. This is especially useful for identifying where significant innovation is occurring across various sectors, as it provides a high-level overview of emerging technologies. The **CPC** and **IPC** systems share a similar hierarchical structure, allowing for easy alignment between the two and facilitating comparative analysis. By using the **WIPO IPC Concordance Table**, we can focus on identifying key technological areas where substantial innovation is happening, without being bogged down by the complexity of detailed subclass classifications. By relying on the WIPO IPC Concordance Table, we avoid the challenges of generalising classifications on our own, especially given the complexities involved in determining the specific applications of technologies.

The grouping of **IPC** subclasses into macrofields enables a clearer understanding of global innovation dynamics and highlights the broader trends shaping the future of technology. This approach provides a more manageable and interpretable perspective, making it easier to track the development of emerging fields across different sectors. The macrofields help identify where innovation is most concentrated and which technologies are likely to drive future industrial shifts.

An important distinction in this context is the **CPC** "Y" class, which specifically addresses technologies related to climate change mitigation and adaptation. While the **WIPO IPC Technology Concordance Table** does not include this "Y" class, we think it might be crucial to treat these climate-related technologies separately, given their growing importance in global sustainability efforts. By doing so, we ensure that technologies such as clean energy solutions, sustainable practices, and climate resilience are clearly recognised as a unique category, distinct from other technological advancements.

Incorporating the **WIPO IPC Technology Concordance Table** into our analysis allows for a structured and strategic overview of global technological trends. It not only enhances our understanding of innovation at a macro level but also ensures that critical fields receive the focused attention they deserve. This approach provides a comprehensive view of the emerging technologies that are shaping the future and positions policymakers and industry leaders to better navigate and respond to these global shifts.


In [ ]:
def clean_IPC(x):
    x = x.replace("/", "").replace("%", "").replace(" ", "")
    x = x[:4]
    return x

In [ ]:
url = "https://www.wipo.int/documents/2948119/3215563/ipc_technology.xlsx"
techno_df = pd.read_excel(
    url,
    skiprows=6,
    usecols=['Field_number', 'Sector_en', 'Field_en', 'IPC_code'],
    converters={"IPC_code": clean_IPC},
)
techno_df.columns = ['Field Nb', 'Sector', 'Field', 'IPC']

In [ ]:
techno_df = (techno_df.groupby(["Field Nb", "Sector", "Field"])["IPC"].agg(", ".join).reset_index())
techno_df

In [ ]:
ipc_to_field_mapping = {}
for _, row in techno_df.iterrows():
    for ipc_code in row['IPC'].split(", "):
        ipc_to_field_mapping[ipc_code.strip()] = row['Field']

def assign_field(cpc_prefix):
    if cpc_prefix in ipc_to_field_mapping:
        return ipc_to_field_mapping[cpc_prefix]
    if cpc_prefix.startswith("Y02"):
        return "Climate change mitigation"
    return "UNKNOWN" 

merged_data["Field"] = merged_data["cpc_class_prefix"].apply(assign_field)
merged_data = merged_data[merged_data["Field"] != "UNKNOWN"]
merged_data

With the inventions now grouped into broader macrofields using the **WIPO IPC Technology Concordance Table**, we can analyse the contributions to these fields from applicants originating in the key regions of interest. By examining the distribution of inventions across macrofields and their geographic origins, we gain insights into how different regions are driving innovation in high-priority technological areas.

For this analysis, the inventions are grouped by their assigned macrofields and the ``person_ctry_code`` of the applicants, which represents their country of origin. This grouping allows us to quantify the number of inventions attributed to applicants from the five influential regions, **United States (US)**, **Japan (JP)**, **China (CN)**, **South Korea (KR)**, and the **European Patent Convention (EP)** contracting states, within each macrofield.


In [ ]:
cpc_country_analysis = (
    merged_data
    .groupby(['Field', 'person_ctry_code'])
    .agg(unique_families=('docdb_family_id', 'nunique'))  
    .reset_index()
    .rename(columns={'unique_families': 'invention_count'})  
    .sort_values(by='Field', ascending=False)
)

cpc_country_analysis

## The contribution of each Country

To better understand the distribution of inventive activity across regions, we calculate the percentage contribution of each country to the total number of inventions within each macrofield. This approach provides a more balanced view of regional influence by contextualising each country’s inventive output relative to the total activity in a given technological field. By examining contributions as percentages, we gain a clearer understanding of how different regions are driving innovation in growing sectors.

The analysis begins by determining the total number of inventions within each macrofield, aggregating the invention counts for all contributing regions. This total serves as the basis for calculating the relative contributions of individual countries. Once these totals are established, they are merged back into the dataset to allow for comparison with the specific contributions of each region. For each country, its percentage contribution is derived by dividing its invention count by the total number of inventions in the macrofield. This reveals the proportion of innovation activity attributable to each region, offering insights into their relative influence and participation in these fields.

The resulting dataset is sorted to highlight the leading contributors within each macrofield. By organising the data in descending order of percentage contribution, we can identify which countries or regions are most active in driving innovation in specific technological areas. This approach normalises the data, enabling a more meaningful comparison of regional contributions irrespective of their overall size or patenting activity.

This percentage-based analysis provides valuable insights into global innovation dynamics. It highlights fields where certain regions demonstrate strong leadership and others where their presence is comparatively limited. For policymakers, understanding these contributions in proportional terms allows for a better assessment of strategic priorities and the identification of sectors where additional investment or collaboration may be needed. Furthermore, this perspective facilitates monitoring of trends over time, particularly in fast-evolving or emerging technologies, providing a clearer understanding of the global innovation landscape.

In [ ]:
cpc_class_totals = (
    cpc_country_analysis
    .groupby('Field', as_index=False)
    .agg({'invention_count': 'sum'})
    .rename(columns={'invention_count': 'total_count'})
)

cpc_class_percentage_analysis = cpc_country_analysis.merge(
    cpc_class_totals,
    on='Field'
)

cpc_class_percentage_analysis['percentage'] = (
    cpc_class_percentage_analysis['invention_count'] /
    cpc_class_percentage_analysis['total_count'] * 100
)


cpc_class_percentage_analysis = cpc_class_percentage_analysis.sort_values(
    by=['Field', 'percentage'], ascending=[True, False]
)

cpc_class_percentage_analysis

To identify the technological fields where each region demonstrates leadership, we analysed the percentage contributions of countries to each macrofield. For each field, the country with the highest percentage of unique patent families was identified as the leading contributor in that area. This analysis provided a comprehensive list of macrofields where the **United States (US)**, **China (CN)**, **European Patent Convention countries (EP)**, **Japan (JP)**, and **South Korea (KR)** are the dominant contributors. By isolating the macrofields led by each region, this approach offers valuable insights into their respective areas of technological strength and focus. The results highlight the competitive positioning of these regions in key innovation areas and reflect the diversity in their technological priorities.


In [ ]:
leaders_by_field = (
    cpc_class_percentage_analysis
    .loc[cpc_class_percentage_analysis.groupby('Field')['percentage'].idxmax()]
)


fields_us_leading = leaders_by_field[leaders_by_field['person_ctry_code'] == 'US']['Field'].tolist()
fields_cn_leading = leaders_by_field[leaders_by_field['person_ctry_code'] == 'CN']['Field'].tolist()
fields_ep_leading = leaders_by_field[leaders_by_field['person_ctry_code'] == 'EP']['Field'].tolist()
fields_jp_leading = leaders_by_field[leaders_by_field['person_ctry_code'] == 'JP']['Field'].tolist()
fields_kr_leading = leaders_by_field[leaders_by_field['person_ctry_code'] == 'KR']['Field'].tolist()

print("Fields where US is leading:", fields_us_leading)
print("Fields where CN is leading:", fields_cn_leading)
print("Fields where EP is leading:", fields_ep_leading)
print("Fields where JP is leading:", fields_jp_leading)
print("Fields where KR is leading:", fields_kr_leading)

The analysis of technological leadership across macrofields reveals distinct patterns in regional contributions to innovation. The **United States (US)** emerges as the leading contributor in a wide range of high-impact and emerging fields. These include biotechnology, computer technology, digital communication, medical technology, micro-structural and nano-technology, organic fine chemistry, pharmaceuticals, and other special machines. This breadth underscores the US’s strong presence in both foundational and advanced technological domains, reflecting its sustained investment in R&D and its global leadership in innovation-intensive sectors.

In contrast, the analysis does not identify any macrofields where **China (CN)** or **European Patent Convention countries (EP)** are the leading contributors. This absence could point to a more balanced distribution of inventive activity across fields for these regions or may indicate areas where these regions are contributing significantly but not dominating.

**Japan (JP)** demonstrates leadership in several specialised technological areas, including electrical machinery, apparatus, and energy, IT methods for management, materials and metallurgy, surface technology and coating, and technologies or applications for mitigation or adaptation against climate change. This highlights Japan’s strength in industrial and environmental technologies, aligning with its longstanding focus on advanced manufacturing and sustainable innovation.


### Adding time

The next step in the analysis focuses on examining the distribution of inventions over time. By analysing the number of unique patent families per year, grouped by macrofield and the applicant's country of origin, we aim to understand the temporal dynamics of innovation. This approach allows us to identify trends in inventive activity across different regions and technological fields, shedding light on whether specific countries are increasing or decreasing their contributions to key areas over the years.

For this purpose, we calculate the count of unique patent families (**`docdb_family_id`**) for each combination of macrofield, publication year (**`earliest_publn_year`**), and applicant country (**`person_ctry_code`**). This temporal breakdown highlights the yearly distribution of inventions and provides a detailed view of how technological contributions evolve within and across fields.

The resulting dataset is organised by field, publication year, and country to facilitate trend analysis. This temporal perspective helps uncover patterns such as periods of rapid innovation, shifts in regional dominance, or the emergence of new technological priorities in specific macrofields. These insights are critical for understanding the dynamics of global innovation and for identifying potential opportunities for investment or collaboration.


In [ ]:
families_by_year_country_field = (
    merged_data
    .groupby(['Field', 'earliest_publn_year', 'person_ctry_code'], as_index=False)
    .agg({'docdb_family_id': 'nunique'})  
    .rename(columns={'docdb_family_id': 'unique_family_count'})  
)

families_by_year_country_field = families_by_year_country_field.sort_values(
    by=['Field', 'earliest_publn_year', 'person_ctry_code']
)

families_by_year_country_field

In [ ]:
families_by_year_field = (
    families_by_year_country_field
    .groupby(['Field', 'earliest_publn_year'], as_index=False)
    .agg({'unique_family_count': 'sum'}) 
)

families_by_year_field = families_by_year_field.sort_values(
    by=['Field', 'earliest_publn_year']
)

families_by_year_field

In [ ]:
unique_years = merged_data['earliest_publn_year'].unique()
print(unique_years)


## Identification of the _Emerging_ Fields

Before continuing with this preliminary analysis, we could try applying a definition of emerging fields by identifying them through the growth trend of innovation intensity within the broader technological fields. This would involve focusing on fields that are experiencing rapid development and increasing activity, which can be tracked through patent trends and the volume of new filings. By looking at these trends, we can pinpoint areas that are gaining momentum and might represent emerging sectors in the global innovation landscape. 
It is important to note that while we have observed a significant number of inventions in the fields we are analysing, this does not necessarily mean they are emerging sectors. These fields are undoubtedly characterised by novelty, as we are focusing on high-value inventions, which are patent families filed across multiple countries. The fact that these inventions are new adds to their transformative potential. They have the capacity to drive substantial change, as they often represent breakthroughs in technology and innovation, with applications that could reshape industries and societies.

However, the presence of many inventions in recent years does not automatically qualify these fields as emerging sectors. *Emerging sectors* are typically characterised not only by the volume of innovations but also by their potential for future growth and market disruption. To identify true emerging sectors, we must look at the trends over time. Some sectors may have consistently seen a high volume of inventions, and their dynamism could be more evolutionary rather than revolutionary. Rapid evolution in these sectors may simply reflect the ongoing progression of established technologies rather than the rise of entirely new, transformative areas.

In the context of growing sectors, we might observe significant levels of activity due to the continuous evolution of existing technologies. These sectors can exhibit high growth and dynamism, where players’ market shares change drastically, often due to technological advancements or shifts in market demand. Such sectors are not necessarily emerging, but they can still capture a large share of economic growth due to their established nature and ongoing developments.

Thus, while we observe high levels of invention activity, we need to carefully assess whether these fields truly represent emerging sectors or whether they are growing sectors with substantial but stable innovation over time. Growing sectors may exhibit high growth and dynamism, but they do not necessarily represent entirely new or disruptive technological frontiers. *Emerging sectors*, on the other hand, often demonstrate a sharper and more accelerated growth trajectory, reflecting the introduction of highly novel and potentially disruptive technologies.

Disruptive inventions tend to act as catalysts within their respective fields, sparking a chain reaction of related innovations. This occurs because groundbreaking technologies often uncover previously unexplored opportunities or address fundamental challenges in entirely new ways. These inventions can redefine the parameters of a field, leading to the development of complementary technologies, refinements, and derivative applications. Additionally, the excitement and potential profitability surrounding such innovations attract increased investment, talent, and research efforts, further accelerating invention activity. As a result, emerging sectors not only reflect technological novelty but also signal areas where industries are being fundamentally reshaped.


Analysing the trends in unique patent families across different fields can provide valuable insights into identifying emerging technologies. By visualising the growth trajectories of patenting activity over time, it becomes possible to distinguish fields that exhibit sudden and sharp increases in invention activity from those with more stable or moderate growth patterns. Fields with accelerated growth often signal emerging trends, as they may reflect the impact of disruptive technologies or heightened interest in areas of significant technological novelty.

The involvement of multiple countries in such trends further highlights the global competitiveness and strategic importance of these fields. Emerging sectors typically display not only rapid growth but also an increasing concentration of efforts and resources, indicating their potential to redefine industries or create entirely new markets. However, to accurately assess the level of effort and resource allocation in these fields, a more detailed analysis is required. This would involve examining the geographic distribution of inventors, the presence of high-value inventions, and the dynamics of collaboration between key stakeholders in these fields.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set Seaborn style
sns.set(style="whitegrid", palette="muted")

fields = families_by_year_country_field['Field'].unique()

# Define color scheme
epo_red = '#D93317'
secondary_colors = {
    'CN': '#1D7A73',  # China - Turquoise
    'JP': '#013B79',  # Japan - Blue
    'KR': '#7F830B',  # South Korea - Lime
    'US': '#FF6800',  # US - Orange
}


# Adjust figure size for multiple fields
plt.figure(figsize=(16, 8))

for field in fields:
    # Create subplots with two axes (left for country breakdown, right for total trend)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Filter the data for the current field
    field_data_country = families_by_year_country_field[families_by_year_country_field['Field'] == field]
    field_data_total = families_by_year_field[families_by_year_field['Field'] == field]
    
    # Left subplot: trends by country
    for country, color in secondary_colors.items():
        country_data = field_data_country[field_data_country['person_ctry_code'] == country]
        sns.lineplot(
            data=country_data,
            x='earliest_publn_year',
            y='unique_family_count',
            label=country,
            marker='o',
            lw=1,
            ax=ax1,
            color=color
        )
    
    # Add EP country in EPO Red
    ep_data = field_data_country[field_data_country['person_ctry_code'] == 'EP']
    sns.lineplot(
        data=ep_data,
        x='earliest_publn_year',
        y='unique_family_count',
        label='EP',
        marker='o',
        lw=1,
        ax=ax1,
        color=epo_red
    )
    
    # Right subplot: total trend across countries (use grey for total trend)
    sns.lineplot(
        data=field_data_total,
        x='earliest_publn_year',
        y='unique_family_count',
        marker='o',
        color=grey_color,  # Grey color for total trend
        lw=1,  # Thinner line for total trend
        ax=ax2
    )
    
    # Add a smooth line for the total trend (right plot)
    sns.regplot(
        data=field_data_total,
        x='earliest_publn_year',
        y='unique_family_count',
        scatter=False,
        ax=ax2,
        color=grey_color,
        line_kws={'color': grey_color, 'lw': 1, 'ls': '--'}  # Thinner dashed grey line
    )
    
    # Titles and labels
    ax1.set_title(f'Trends in {field} (By Country)', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Publication Year', fontsize=14)
    ax1.set_ylabel('Inventions', fontsize=14)
    ax1.legend(title='Country', fontsize=12)
    ax1.grid(True, linestyle='--', alpha=0.6)
    
    ax2.set_title(f'Trends in {field} (Total)', fontsize=16, fontweight='bold')
    ax2.set_xlabel('Publication Year', fontsize=14)
    ax2.set_ylabel('Inventions', fontsize=14)
    ax2.grid(True, linestyle='--', alpha=0.6)
    
    # Customize font size and add annotations for emphasis
    for tick in ax1.get_xticklabels():
        tick.set_fontsize(12)
    for tick in ax1.get_yticklabels():
        tick.set_fontsize(12)
    
    # Adjust layout for better spacing
    plt.tight_layout()
    
    # Show the plot
    plt.show()


When analysing the trends across different patent fields, it becomes evident that there are distinct growth patterns, each offering valuable insights into emerging and growing technologies.

In the field of **Basic Materials Chemistry**, we observe moderate growth across the countries being analysed, particularly in **KR** and **CN**. However, there is nothing exceptionally rapid in the growth rates for these countries. Looking at the *general trend*, it is clear that there has been a consistent increase in innovations within this field over the years, suggesting a steady but ongoing progression in this area.


In contrast, the **Biotechnology** field demonstrates a more pronounced and steep growth trajectory, particularly in the **US**, where innovation activity is markedly higher compared to other countries. The *general trend* shows significant growth, indicating that the **US** is likely leading the charge in this domain, while other regions have not experienced similar growth patterns in this specific field.


The **Computer Technology** field shows strong growth starting in **2018** across all countries considered, with the **US** maintaining its leadership position. The **general trend** indicates consistent growth, with a bit more variation in the range of differences between countries. While the **US** leads, other countries are also experiencing growth, but the **US** remains ahead in terms of total innovations.

A similar trend can be observed in **Digital Communication**, where **CN** has shown significant progress. Since **2020**, the gap between **CN** and the **US** has narrowed, with the number of inventions in this field becoming nearly identical in both regions. The **general trend** in **Digital Communication** shows consistent growth, especially in **CN**, making it clear that this country is catching up with the **US** in terms of technological advancements.


In the field of **Electrical Machinery, Apparatus, and Energy**, growth has been moderate, with **KR** and **CN** standing out as countries that have made significant strides in increasing their patent filings, while other countries have shown more stable growth. The **general growth** is **less steep** compared to the previous fields.

For the field of **IT Methods for Management**, we see a general upward trend, albeit with some fluctuations, indicating steady but not extraordinary growth across the countries. Again, the **general trend** is **less rapid and decisive** than the previous fields.

The field of **Materials and Metallurgy** exhibits stability, with **JP** emerging as the clear leader in this domain, while other countries remain at much lower levels.
In **Medical Technology**, a significant and rapid growth has been evident since 2018, particularly in the **US**, underscoring the increasing global importance of this field.
On the other hand, **Microstructural and Nanotechnology** shows an overall decline, although the number of inventions remains significant. This **is particularly visible** in the **general trend**.


Fields such as **Organic Fine Chemistry** and **Pharmaceuticals** show stability, while **Other Special Machines** has seen growth that has since stabilised when looking at individual countries. Even though the general trend remains upward, the growth has plateaued.
Similarly, **Surface Technology and Coating** also remains steady, and the general trend is **not** growing as rapidly as other fields; in fact, it shows a slight decline.
The field of **Technologies or Applications for Mitigation or Adaptation Against Climate Change** has shown stability in the **Europe** area, **JP**, and the **US**, but with **KR** on the rise and **CN** making notable strides since 2018. Indeed, the **general trend** is a clear indicator of increased involvement and innovation activity in this field over the last 10 years.


What we can conclude is that *it is very important to take the time factor into account* in order to determine whether the intensity is simply due to the field growing steadily over a long period, or if the interest in innovating within this field is indeed on the rise.



### Relative Growth over Time

An alternative approach for identifying emerging fields could be to calculate the relative growth over the entire 10-year period under analysis. This method would offer a broader perspective on long-term trends and might reveal sectors that have demonstrated consistent growth over time. By examining relative growth across all ten years, we could potentially identify fields where innovation is not only increasing but also maintaining a steady momentum.

However, this approach comes with both advantages and limitations. On the one hand, it helps highlight fields with sustained growth, which could be indicative of emerging sectors with lasting potential. Such an analysis could complement more recent trends and provide a more comprehensive understanding of long-term innovation patterns.

On the other hand, this approach might overlook shorter-term fluctuations or fail to capture emerging fields that exhibit rapid growth over just a few years, but not consistently across the entire period. Additionally, sectors with early-stage innovations might show slower relative growth in the initial years, only to experience a surge later on, which could be missed by focusing on the entire period.

Thus, while calculating relative growth over a 10-year period could provide valuable insights, it is important to carefully weigh its benefits against its limitations and consider it as one part of a broader analysis to identify truly emerging sectors.


In [ ]:
aggregated_data = families_by_year_country_field.groupby(['Field', 'earliest_publn_year'])['unique_family_count'].sum().reset_index()

fields = aggregated_data['Field'].unique()
growth_rates = {}

for field in fields:
    field_data = aggregated_data[aggregated_data['Field'] == field]
    first_year = field_data['earliest_publn_year'].min()
    last_year = field_data['earliest_publn_year'].max()
    
    first_count = field_data[field_data['earliest_publn_year'] == first_year]['unique_family_count'].values[0]
    last_count = field_data[field_data['earliest_publn_year'] == last_year]['unique_family_count'].values[0]
    
    if first_count > 0:
        relative_growth = ((last_count - first_count) / first_count) * 100 
        growth_rates[field] = relative_growth

growth_df = pd.DataFrame(list(growth_rates.items()), columns=['Field', 'Relative Growth (%)'])

growth_df = growth_df.sort_values(by='Relative Growth (%)', ascending=False)

growth_df



The results reveal that **Medical technology** and **Computer technology** are the sectors with the highest relative growth, showing increases of over 490%, indicating a strong rise in patent activity in these areas. **Other special machines** and **Digital communication** also exhibit significant growth, with percentages of 333% and 193% respectively, highlighting substantial developments in areas such as special machinery and digital communication. However, **Micro-structural and nano-technology** shows a decrease of -46.92%, suggesting a possible maturation or consolidation of technology in this field. These results reflect the evolution of emerging technologies and provide valuable insights into sectors with the highest innovation potential, while other areas may require further investigation to understand the dynamics driving their growth.

In [ ]:
plt.figure(figsize=(12, 8))
plt.barh(growth_df['Field'], growth_df['Relative Growth (%)'], color='#59D6CC')
plt.xlabel('Relative Growth (%) (2013-2022)')
plt.ylabel('Field')
plt.title('Relative Growth in Unique Patent Families (2013-2022)')
plt.gca().invert_yaxis() 
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Looking at the relative growth rates, we can see that the sectors **Medical Technology**, **Computer Technology**, and **Digital Communication** all exhibit high growth rates. Interestingly, **Other Special Machines** is also identified as having a high growth rate. While the trend graph for this sector may not appear particularly steep, the fact that it has experienced a consistent and substantial growth over time has led to its identification with a high relative growth rate. This highlights how sustained, steady growth can sometimes be just as significant as rapid increases, particularly when considering long-term trends.


### Compound Annual Growth Rate (CAGR)


Another approach that could be considered for analysing growth patterns across different fields is the use of the Compound Annual Growth Rate (CAGR). To gain a more nuanced understanding of growth trends, we computed the CAGR for each sector over the 10-year period from 2013 to 2022. The CAGR is a useful metric for analysing the growth of a sector, country, or other parameter over time, as it provides a "smoothed" measure of growth. By excluding the effects of short-term fluctuations, it enables a more reliable assessment of the average annual growth rate over the observation period.

The calculation process began with aggregating the data by Field and earliest_publn_year, summing the unique family counts for each combination. Next, we calculated the CAGR for each field by comparing the values at the start (2013) and end (2022) of the period, taking into account the number of years in between. This method allows for a clear understanding of both the relative growth rate and the total increase in patent families.

The formula used for CAGR is:

$$
\text{CAGR} = \left(\frac{\text{Final Count}}{\text{Initial Count}}\right)^{\frac{1}{\text{Number of Years}}} - 1
$$

This formula correctly calculates the average annual growth rate for each field over the time period under consideration.


To ensure accuracy, we made sure that the initial count was non-zero, which prevents any issues with the calculation. The resulting data was then stored and organised into a DataFrame, sorted by CAGR in descending order. This approach helps highlight the sectors with the highest growth, offering a clearer picture of the areas with the most significant long-term growth trends.

While the CAGR method provides a useful long-term growth perspective, it is important to consider the potential limitations of this approach. Specifically, the CAGR is a simplification that does not account for potential fluctuations or variations within the years of the period, which may mask shorter-term trends. As such, while the CAGR highlights sustained growth, it is still valuable to combine this analysis with other methods, such as examining year-to-year changes or more granular data, to provide a more comprehensive understanding of sector dynamics.


In [ ]:
cagr_list = []

for field, group in aggregated_data.groupby('Field'):
    group = group.sort_values(by='earliest_publn_year')  
    first_count = group.iloc[0]['unique_family_count']   
    last_count = group.iloc[-1]['unique_family_count']   
    n_years = group['earliest_publn_year'].max() - group['earliest_publn_year'].min()  
    
    if first_count > 0 and n_years > 0:
        cagr = (last_count / first_count) ** (1 / n_years) - 1
        cagr_list.append({
            'Field': field,
            'Initial Year': group['earliest_publn_year'].min(),
            'Final Year': group['earliest_publn_year'].max(),
            'CAGR (%)': cagr * 100,
            'Total Growth': last_count - first_count,
            'Initial Count': first_count,
            'Final Count': last_count
        })

cagr_df = pd.DataFrame(cagr_list)
cagr_df = cagr_df.sort_values(by='CAGR (%)', ascending=False)
print("CAGR per Field (Aggregato):")
cagr_df

In [ ]:
plt.figure(figsize=(14, 10))
plt.barh(cagr_df['Field'], cagr_df['CAGR (%)'], color='#BEC510')
plt.xlabel('CAGR (%)')
plt.ylabel('Field')
plt.title('CAGR for All Fields (Aggregated by Field, 2013-2022)')
plt.gca().invert_yaxis()  
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Looking at the results of the **Compound Annual Growth Rate (CAGR)** analysis, we can identify several key trends across different fields. **Medical Technology** and **Computer Technology** both exhibit exceptionally high growth, with **CAGR values of 21.99%** and **21.91%** respectively, reflecting substantial increases in patent families over the period from 2013 to 2022. Similarly, **Other Special Machines** shows strong growth at **17.69%**, indicating a consistent rise in activity, despite not having as steep a growth curve as the leading fields. **Digital Communication** follows closely with a **CAGR of 12.71%**, driven primarily by the significant increase in the number of inventions in the sector. 

Fields such as **Biotechnology** and **IT Methods for Management** also demonstrate solid growth, though at a somewhat slower pace, with **CAGR values of 10.24%** and **9.34%** respectively. On the other hand, **Surface Technology and Coating** and **Basic Materials Chemistry** show moderate growth at **6.67%** and **5.95%**, indicating steady but less rapid advancements.

However, there are sectors that have seen more modest growth or even decline. **Electrical Machinery, Apparatus, and Energy** has a **CAGR of 5.60%**, while **Materials and Metallurgy** shows a more limited growth of **5.09%**. **Pharmaceuticals** and **Organic Fine Chemistry** show lower growth rates, at **3.00%** and **1.94%**, with **Organic Fine Chemistry** being the slowest-growing sector. Notably, **Micro-structural and Nano-technology** is the only field to exhibit a negative **CAGR of -6.80%**, reflecting a decline in patenting activity over the past decade.


These results reveal varying growth patterns across sectors, with some emerging technologies demonstrating significant increases in innovation intensity, while others show more stable or even declining trends.


Connected to the previous approach, we calculated the Compound Annual Growth Rate (CAGR) for each field and country to gain a more detailed understanding of growth trends over the long term. This approach provides a measure of the average annual growth rate, which is particularly useful for comparing growth rates across different sectors and countries while accounting for short-term fluctuations. In this way, we can identify sectors and countries that have experienced sustained and more stable growth over the years, offering a clearer view of innovation areas that may continue to develop in the future.

In [ ]:
cagr_list = []

for (field, country), group in families_by_year_country_field.groupby(['Field', 'person_ctry_code']):
    group = group.sort_values(by='earliest_publn_year')  
    first_count = group.iloc[0]['unique_family_count']   
    last_count = group.iloc[-1]['unique_family_count']   
    n_years = group['earliest_publn_year'].max() - group['earliest_publn_year'].min()  
    if first_count > 0 and n_years > 0:
        cagr = (last_count / first_count) ** (1 / n_years) - 1
        cagr_list.append({
            'Field': field,
            'Country': country,
            'Initial Year': group['earliest_publn_year'].min(),
            'Final Year': group['earliest_publn_year'].max(),
            'CAGR (%)': cagr * 100
        })

cagr_df = pd.DataFrame(cagr_list)
cagr_df = cagr_df.sort_values(by='CAGR (%)', ascending=False)
print("CAGR per Field e Paese:")
cagr_df

In [ ]:
plt.figure(figsize=(15, 12))
plt.barh(
    cagr_df['Field'] + " (" + cagr_df['Country'] + ")", 
    cagr_df['CAGR (%)'], 
    color='#BEC510'
)
plt.xlabel('CAGR (%)')
plt.ylabel('Field (Country)')
plt.title('CAGR for All Fields and Countries (2013-2022)')
plt.gca().invert_yaxis()  
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

When we examine the **Compound Annual Growth Rate (CAGR)** by country, we can identify which nations have experienced the most significant growth across various fields. For instance, **Other Special Machines** shows the highest CAGR of **44.34%**, driven largely by the substantial increase in patenting activity in **China** (**CN**). Similarly, **Medical Technology** has also seen significant growth, with **South Korea** (**KR**) achieving a **CAGR of 40.64%** and **China** (**CN**) at **39.38%**. **Computer Technology** in **China** shows a notable **CAGR of 34.97%**, reflecting the country's dominant role in this sector. Other fields such as **IT Methods for Management** and **Biotechnology** also show strong performance in **China**, with **CAGR values of 33.19%** and **23.37%** respectively.

On the other hand, there are fields that have experienced slower growth or even a decline in certain countries. For example, **Micro-structural and Nano-technology** shows a **negative CAGR** across multiple countries, including **Japan** (**JP**), **South Korea** (**KR**), the **United States** (**US**), and **Europe (EP)**, with **South Korea** seeing the largest decline at **-7.71%**.

This analysis highlights the strong growth in certain countries, particularly **China**, which has played a significant role in the overall growth of sectors like **Medical Technology**, **Computer Technology**, and **Other Special Machines**. The growth in **Other Special Machines** is especially notable, as the **CAGR is largely driven by China**, reflecting the rapid expansion of manufacturing and industrial sectors in the country.


## Conclusion


In conclusion, our preliminary analysis has shown that there are various ways to identify emerging fields. Looking at sectors with high innovation intensity, it becomes clear that Europe is not leading in any of the identified fields. For instance, when examining the field of Computer Technology, Europe’s contribution is the weakest among the major states, although it remains close to the levels seen in Japan and South Korea. In Digital Communication, the clear leaders are China and the United States, while Europe, on the other hand, is on par with Japan, showing a similar level of contribution. In the field of Medical Technology, European contracting countries rank second, yet the United States remains the undisputed leader in this domain.

This highlights the critical need for Europe to enhance its investments in these high-potential areas to ensure it remains competitive globally. While Europe has made strides in emerging sectors, it must ramp up efforts to catch up with the innovation leaders. Identifying and addressing these innovation gaps is essential for shaping Europe’s future R&D strategy and ensuring its leadership in the global innovation landscape. We have explored one approach to begin investigating which fields are emerging fields.

# Global Applicants in Emerging Sectors

## Introduction


After identifying potential emerging fields, it makes sense to focus on the main applicants in these fields during the period 2013–2022. We will also report on their geographical distribution to better understand where the inventive activities originate. To make the analysis more concise, we have decided to focus on three of the fields previously examined: Computer Technology, Medical Technology, and Digital Communication, where the trends in inventions have shown consistent growth. Additionally, these fields have demonstrated notable relative growth and strong Compound Annual Growth Rates (CAGR), indicating their prominence as emerging sectors.

We divide the data into different fields because we want to gain insights into each individual field.

In [ ]:
computer_technology_data = merged_data[merged_data['Field'] == 'Computer technology']
medical_technology_data = merged_data[merged_data['Field'] == 'Medical technology']
digital_communication_data = merged_data[merged_data['Field'] == 'Digital communication']

## Top Applicants

It is particularly interesting to examine the top 25 applicants within the emerging fields, as this allows us to identify the key players driving innovation in these areas. By focusing on these applicants and considering their performance over the entire period, we can gain valuable insights into their contributions, strategies, and the dynamics shaping each field over time.

### Computer Technology

In [ ]:
name_country_combinations_ct = (
    computer_technology_data.groupby(['psn_name', 'person_ctry_code'])['docdb_family_id']
    .nunique()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

top_applicants_with_country_ct = (
    name_country_combinations_ct
    .head(25)
    .reset_index(drop=True)
)

top_applicants_with_country_ct

In [ ]:
total_count_ct = name_country_combinations_ct ['count'].sum()

top_25_total_count_ct = top_applicants_with_country_ct['count'].sum()

top_25_share_ct = (top_25_total_count_ct / total_count_ct) * 100

print(f"The top 25 applicants in Computer Technology contribute {top_25_share_ct:.2f}% of the total.")


The top 25 applicants together account for 26.26% of all International Patent Families (IPFs) in the **Computer Technology** field. The United States and China are each represented by 7 companies, while Japan is represented by 5, Europe by 4, and South Korea by 2. 

*Samsung* is the clear leader in **Computer Technology**, with over 5,500 IPFs, corresponding to 18.28% of all inventions in the field between 2013 and 2022. *Samsung*(KR) is followed by *Microsoft* (US) and *Huawei*(CN), which contribute 9.1% and 6.6%, respectively. After them, another South Korean company, *LG Electronics*, and *Google* (US) follow closely.


In [ ]:
plt.figure(figsize=(28, 10))
bars = plt.barh(
    top_applicants_with_country_ct["psn_name"] + " (" + top_applicants_with_country_ct["person_ctry_code"] + ")",
    top_applicants_with_country_ct["count"],
    color="#FF8200"
)


for bar in bars:
    plt.text(
        bar.get_width() + 100,  
        bar.get_y() + bar.get_height() / 2,  
        f"{int(bar.get_width()):,}",  
        va="center", fontsize=9
    )


plt.xlabel("Number of Applications", fontsize=12)
plt.title("Top 25 Applicants in Computer Technology", fontsize=14)
plt.gca().invert_yaxis()  
plt.grid(False)
plt.tight_layout()

### Medical Technology

In [ ]:
name_country_combinations_mt = (
    medical_technology_data.groupby(['psn_name', 'person_ctry_code'])['docdb_family_id']
    .nunique()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

top_applicants_with_country_mt = (
    name_country_combinations_mt
    .head(25)
    .reset_index(drop=True)
)


top_applicants_with_country_mt


In [ ]:
total_count_mt = name_country_combinations_mt ['count'].sum()

top_25_total_count_mt = top_applicants_with_country_mt['count'].sum()

top_25_share_mt = (top_25_total_count_mt / total_count_mt) * 100

print(f"The top 25 applicants in Medical Technology contribute {top_25_share_mt:.2f}% of the total.")


The top 25 applicants in **Medical Technology** account for 16.87% of all inventions in the field. *Philips Electronics* (EP) is the leader, with just over 601 inventions, followed by *Fujifilm* (JP) and *Siemens Healthcare* (EP). The leader contributes 22.3% of the total families in this field, while *Fujifilm* contributes 9% and *Siemens Healthcare* contributes 7%. Notably, 3 companies in the top 5 are European, 10 are Japanese, 8 are American, and 2 are Chinese and South Korean.

In [ ]:
plt.figure(figsize=(28, 10))
bars = plt.barh(
    top_applicants_with_country_mt["psn_name"] + " (" + top_applicants_with_country_mt["person_ctry_code"] + ")",
    top_applicants_with_country_mt["count"],
    color="#0262CA"
)


for bar in bars:
    plt.text(
        bar.get_width() + 10,  
        bar.get_y() + bar.get_height() / 2,  
        f"{int(bar.get_width()):,}",  
        va="center", fontsize=9
    )


plt.xlabel("Number of Applications", fontsize=12)
plt.title("Top 25 Applicants in Medical Technology", fontsize=14)
plt.gca().invert_yaxis()  
plt.grid(False)
plt.tight_layout()

### Digital Communication

In [ ]:
name_country_combinations_dc = (
    digital_communication_data.groupby(['psn_name', 'person_ctry_code'])['docdb_family_id']
    .nunique()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

top_applicants_with_country_dc = (
    name_country_combinations_dc
    .head(25)
    .reset_index(drop=True)
)

top_applicants_with_country_dc

In [ ]:
total_count_dc = name_country_combinations_dc ['count'].sum()

top_25_total_count_dc = top_applicants_with_country_dc['count'].sum()

top_25_share_dc = (top_25_total_count_dc / total_count_dc) * 100

print(f"The top 25 applicants in Digital Communication contribute {top_25_share_dc:.2f}% of the total.")


The top 25 applicants in **Digital Communication** account for a significant 44.96% of all inventions in the field. *Qualcomm* stands out as the clear leader with 9,475 inventions, contributing 15.8% of all IPFs in this field. Following closely behind is *Huawei* (CN), contributing 14% of the total inventions, and *LG Electronics* (KR), with 12%. These three companies are dominant players in this sector, making substantial contributions to the development of digital communication technologies. In terms of geographical representation, the top 25 applicants include 8 Chinese companies, 6 Japanese, 4 from the US and South Korea, and 3 European companies.



In [ ]:
plt.figure(figsize=(24, 10))
bars = plt.barh(
    top_applicants_with_country_dc["psn_name"] + " (" + top_applicants_with_country_dc["person_ctry_code"] + ")",
    top_applicants_with_country_dc["count"],
    color="#337878"
)


for bar in bars:
    plt.text(
        bar.get_width() + 200,  
        bar.get_y() + bar.get_height() / 2,  
        f"{int(bar.get_width()):,}",  
        va="center", fontsize=9
    )


plt.xlabel("Number of Applications", fontsize=12)
plt.title("Top 25 Applicants in Digital Communication", fontsize=14)
plt.gca().invert_yaxis()  
plt.grid(False)
plt.tight_layout()

As can be observed from the charts, the top applicants in each field are remarkably dominant. In the case of **Computer Technology**, the leader, *Samsung*, contributes twice as much as the second applicant, *Microsoft*. Regarding **Medical Technology**, *Philips Electronics* contributes over two times the volume of the second applicant, *Fujifilm*. For **Digital Communication**, the leader, *Qualcomm*, contributes almost the same amount as *Huawei*, which is closely followed by *LG Electronics*. All three are significant contributors to inventions in this field, reflecting their active involvement in driving innovation within **Digital Communication**.

## Evolution of Top Applicants Over Time

### Computer Technology

#### Rationale Behind the Analysis of Top Applicants

The analysis of the top applicants over the periods 2013-2017 and 2018-2022 aims to observe the evolution of innovation and the competitive landscape within the **Computer Technology** field. By splitting the data into two time periods, we can identify companies that have consistently led the field, as well as those who have risen or fallen in rank.

We begin by filtering the data to include only patent families published between 2013 and 2017 and 2018 and 2022. This allows us to focus on the most relevant inventions during these specific periods.

To examine the dynamics between companies, we group the data by the company name and country code, counting the number of patent families for each combination. This gives us insight into which companies were the most active in filing patents during these periods. We then identify the top 25 companies by the number of patent families filed and calculate their share of the total patent families in each period.

Next, we merge the two data sets to compare the top 25 companies in both periods and assess their ranking changes. The `ranking_change` function was applied to determine if a company has moved up, down, or remained in the same position in the rankings between 2013-2017 and 2018-2022.

In [ ]:
period_2013_2017_ct = computer_technology_data[
    (computer_technology_data['earliest_publn_year'] >= 2013) & 
    (computer_technology_data['earliest_publn_year'] <= 2017)
]

period_2018_2022_ct = computer_technology_data[
    (computer_technology_data['earliest_publn_year'] >= 2018) & 
    (computer_technology_data['earliest_publn_year'] <= 2022)
]


name_country_combinations_2013_2017_ct = (
    period_2013_2017_ct.groupby(['psn_name', 'person_ctry_code'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

name_country_combinations_2018_2022_ct = (
    period_2018_2022_ct.groupby(['psn_name', 'person_ctry_code'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)


top_25_2013_2017_ct = name_country_combinations_2013_2017_ct.head(25)
top_25_2018_2022_ct = name_country_combinations_2018_2022_ct.head(25)
top_25_2013_2017_ct = top_25_2013_2017_ct.reset_index(drop=True)
top_25_2018_2022_ct = top_25_2018_2022_ct.reset_index(drop=True)
total_count_2013_2017_ct = top_25_2013_2017_ct['count'].sum()
total_count_2018_2022_ct = top_25_2018_2022_ct['count'].sum()

top_25_2013_2017_ct['share'] = (
    top_25_2013_2017_ct['count'] / total_count_2013_2017_ct * 100
).round(1)
top_25_2018_2022_ct['share'] = (
    top_25_2018_2022_ct['count'] / total_count_2018_2022_ct * 100
).round(1)

merged_top_25_ct = pd.merge(
    top_25_2013_2017_ct,
    top_25_2018_2022_ct,
    left_index=True,
    right_index=True,
    suffixes=('_2013_2017', '_2018_2022')
)

merged_top_25_ct


merged_top_25_ct = merged_top_25_ct.rename(
    columns={
        'psn_name_2013_2017': 'company_2013_2017',
        'psn_name_2018_2022': 'company_2018_2022'
    }
)

merged_top_25_ct['company_2013_2017'] = (
    merged_top_25_ct['company_2013_2017'] + " (" + merged_top_25_ct['person_ctry_code_2013_2017'] + ")"
)
merged_top_25_ct['company_2018_2022'] = (
    merged_top_25_ct['company_2018_2022'] + " (" + merged_top_25_ct['person_ctry_code_2018_2022'] + ")"
)

merged_top_25_ct = merged_top_25_ct.drop(columns=['count_2013_2017', 'count_2018_2022','person_ctry_code_2013_2017', 'person_ctry_code_2018_2022'])

ranking_2018_2022_ct = {
    name: rank for rank, name in enumerate(merged_top_25_ct['company_2018_2022'], start=1)
}
ranking_2013_2017_ct = {
    name: rank for rank, name in enumerate(merged_top_25_ct['company_2013_2017'], start=1)
}

def ranking_change(row):
    company_2018_2022 = row['company_2018_2022']
    company_2013_2017 = row['company_2013_2017']

  
    rank_2013_2017 = ranking_2013_2017_ct.get(company_2018_2022, None)
    rank_2018_2022 = ranking_2018_2022_ct.get(company_2018_2022, None)

    if rank_2013_2017 is None:
        return '+'
    elif rank_2013_2017 == rank_2018_2022:
        return '='
    elif rank_2013_2017 > rank_2018_2022:
        return '+'
    else:
        return '-'

merged_top_25_ct['ranking_change'] = merged_top_25_ct.apply(ranking_change, axis=1)
merged_top_25_ct

#### Observations from the Results

From the resulting table, we can observe the following trends in the **Computer Technology** field:

- **Samsung Electronics (KR)** remains the dominant player in both periods, although its share slightly decreases from 26.3% in 2013-2017 to 18.3% in 2018-2022.
- Other top companies, such as **Microsoft Technology Licensing (US)**, **Apple (US)**, and **LG Electronics (KR)**, have experienced some ranking changes. For example, **Apple** has moved up in rank, whereas **Microsoft** has slightly dropped.
- The **share** of top companies has shifted, with **Samsung** still holding the largest share but with increasing competition from companies like **Huawei**, **Google**, and **Microsoft**.

Notably, the table also highlights the emerging players, such as **Tencent Technology (CN)** and **Beijing Baidu Netcom Science and Technology Co. (CN)**, which have gained significant ground between the two periods.

This analysis offers a glimpse into the dynamics of the **Computer Technology** field, showing which companies are maintaining their competitive edge and which ones are climbing the ranks or losing ground in terms of innovation.


### Medical Technology

We begin by filtering the data to include patent families published between 2013 and 2017 and between 2018 and 2022. This allows us to focus on the relevant inventions in these specific periods. The data is grouped by **company name** and **country code** to identify the most active companies in both periods.

For each period, we calculate the total number of patents filed by the top 25 companies and their corresponding market share. We then merge the data from both periods to compare the top 25 companies from 2013-2017 with those from 2018-2022.

The `ranking_change` function is applied to determine the change in rankings between the two periods. This function helps identify whether a company has moved up (+), moved down (-), or remained in the same position (=) over time.

In [ ]:
period_2013_2017_mt = medical_technology_data[
    (medical_technology_data['earliest_publn_year'] >= 2013) & 
    (medical_technology_data['earliest_publn_year'] <= 2017)
]

period_2018_2022_mt = medical_technology_data[
    (medical_technology_data['earliest_publn_year'] >= 2018) & 
    (medical_technology_data['earliest_publn_year'] <= 2022)
]


name_country_combinations_2013_2017_mt = (
    period_2013_2017_mt.groupby(['psn_name', 'person_ctry_code'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

name_country_combinations_2018_2022_mt = (
    period_2018_2022_mt.groupby(['psn_name', 'person_ctry_code'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)


top_25_2013_2017_mt = name_country_combinations_2013_2017_mt.head(25)
top_25_2018_2022_mt = name_country_combinations_2018_2022_mt.head(25)
top_25_2013_2017_mt = top_25_2013_2017_mt.reset_index(drop=True)
top_25_2018_2022_mt = top_25_2018_2022_mt.reset_index(drop=True)

total_count_2013_2017_mt = top_25_2013_2017_mt['count'].sum()
total_count_2018_2022_mt = top_25_2018_2022_mt['count'].sum()

top_25_2013_2017_mt['share'] = (
    top_25_2013_2017_mt['count'] / total_count_2013_2017_mt * 100
).round(1)
top_25_2018_2022_mt['share'] = (
    top_25_2018_2022_mt['count'] / total_count_2018_2022_mt * 100
).round(1)

merged_top_25_mt = pd.merge(
    top_25_2013_2017_mt,
    top_25_2018_2022_mt,
    left_index=True,
    right_index=True,
    suffixes=('_2013_2017', '_2018_2022')
)




merged_top_25_mt = merged_top_25_mt.rename(
    columns={
        'psn_name_2013_2017': 'company_2013_2017',
        'psn_name_2018_2022': 'company_2018_2022'
    }
)

merged_top_25_mt['company_2013_2017'] = (
    merged_top_25_mt['company_2013_2017'] + " (" + merged_top_25_mt['person_ctry_code_2013_2017'] + ")"
)
merged_top_25_mt['company_2018_2022'] = (
    merged_top_25_mt['company_2018_2022'] + " (" + merged_top_25_mt['person_ctry_code_2018_2022'] + ")"
)

merged_top_25_mt = merged_top_25_mt.drop(columns=['count_2013_2017', 'count_2018_2022','person_ctry_code_2013_2017', 'person_ctry_code_2018_2022'])

ranking_2018_2022_mt = {
    name: rank for rank, name in enumerate(merged_top_25_mt['company_2018_2022'], start=1)
}
ranking_2013_2017_mt = {
    name: rank for rank, name in enumerate(merged_top_25_mt['company_2013_2017'], start=1)
}

def ranking_change(row):
    company_2018_2022 = row['company_2018_2022']
    company_2013_2017 = row['company_2013_2017']

  
    rank_2013_2017 = ranking_2013_2017_mt.get(company_2018_2022, None)
    rank_2018_2022 = ranking_2018_2022_mt.get(company_2018_2022, None)

    if rank_2013_2017 is None:
        return '+'
    elif rank_2013_2017 == rank_2018_2022:
        return '='
    elif rank_2013_2017 > rank_2018_2022:
        return '+'
    else:
        return '-'

merged_top_25_mt['ranking_change'] = merged_top_25_mt.apply(ranking_change, axis=1)

merged_top_25_mt


#### Observations from the Results

From the resulting table, we can observe several key trends in the **Medical Technology** field:

- **Philips Electronics (EP)** maintains a dominant position in both periods, although its share slightly decreases from 26.2% in 2013-2017 to 22.8% in 2018-2022, showing a slight decline in patent filings.
- **Samsung Electronics (KR)**, initially ranked second in 2013-2017, falls to fourth place in 2018-2022. However, its share still increases from 10.2% to 18.3%, indicating a significant rise in its patent activity.
- **Fujifilm (JP)** sees a significant rise in rank, moving from 10th place in 2013-2017 to second place in 2018-2022, reflecting an increase in their market share from 2.6% to 6.4%.
- **HeartFlow (US)**, **Siemens Healthcare (EP)**, and **Roche (EP)** are among the companies that have seen an increase in their ranking, indicating growing innovation in medical technology during the 2018-2022 period.
- Notably, **Medtronic (US)**, **GE (General Electric Company) (US)**, and **Canon (JP)** have seen small shifts, but they remain prominent players in the field.

The ranking changes also highlight the emergence of companies like **People.AI (US)** and **Paige.AI (US)**, which indicate new players contributing to innovations in medical technology.

Overall, this analysis showcases the shifting dynamics in the medical technology sector, with established companies retaining leadership while others are rising to prominence.

### Digital Communication

We begin by filtering the data to include patent families published between 2013 and 2017 and between 2018 and 2022. This allows us to focus on patents relevant to each period. The data is grouped by **company name** and **country code** to highlight the most active companies in each period.

For each period, we calculate the total number of patents filed by the top 25 companies and their corresponding market share. We then merge the data from both periods to compare the top 25 companies from 2013-2017 with those from 2018-2022.

The `ranking_change` function is applied to determine how each company's rank has shifted between the two periods. This function helps us identify whether a company has moved up (+), moved down (-), or remained in the same position (=) over time.


In [ ]:
period_2013_2017_dc = digital_communication_data[
    (digital_communication_data['earliest_publn_year'] >= 2013) & 
    (digital_communication_data['earliest_publn_year'] <= 2017)
]

period_2018_2022_dc = digital_communication_data[
    (digital_communication_data['earliest_publn_year'] >= 2018) & 
    (digital_communication_data['earliest_publn_year'] <= 2022)
]


name_country_combinations_2013_2017_dc = (
    period_2013_2017_dc.groupby(['psn_name', 'person_ctry_code'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

name_country_combinations_2018_2022_dc = (
    period_2018_2022_dc.groupby(['psn_name', 'person_ctry_code'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)


top_25_2013_2017_dc = name_country_combinations_2013_2017_dc.head(25)
top_25_2018_2022_dc = name_country_combinations_2018_2022_dc.head(25)
top_25_2013_2017_dc = top_25_2013_2017_dc.reset_index(drop=True)
top_25_2018_2022_dc = top_25_2018_2022_dc.reset_index(drop=True)

total_count_2013_2017_dc = top_25_2013_2017_dc['count'].sum()
total_count_2018_2022_dc = top_25_2018_2022_dc['count'].sum()

top_25_2013_2017_dc['share'] = (
    top_25_2013_2017_dc['count'] / total_count_2013_2017_dc * 100
).round(1)
top_25_2018_2022_dc['share'] = (
    top_25_2018_2022_dc['count'] / total_count_2018_2022_dc * 100
).round(1)

merged_top_25_dc = pd.merge(
    top_25_2013_2017_dc,
    top_25_2018_2022_dc,
    left_index=True,
    right_index=True,
    suffixes=('_2013_2017', '_2018_2022')
)




merged_top_25_dc = merged_top_25_dc.rename(
    columns={
        'psn_name_2013_2017': 'company_2013_2017',
        'psn_name_2018_2022': 'company_2018_2022'
    }
)

merged_top_25_dc['company_2013_2017'] = (
    merged_top_25_dc['company_2013_2017'] + " (" + merged_top_25_dc['person_ctry_code_2013_2017'] + ")"
)
merged_top_25_dc['company_2018_2022'] = (
    merged_top_25_dc['company_2018_2022'] + " (" + merged_top_25_dc['person_ctry_code_2018_2022'] + ")"
)

merged_top_25_dc = merged_top_25_dc.drop(columns=['count_2013_2017', 'count_2018_2022','person_ctry_code_2013_2017', 'person_ctry_code_2018_2022'])

ranking_2018_2022_dc = {
    name: rank for rank, name in enumerate(merged_top_25_dc['company_2018_2022'], start=1)
}
ranking_2013_2017_dc = {
    name: rank for rank, name in enumerate(merged_top_25_dc['company_2013_2017'], start=1)
}

def ranking_change(row):
    company_2018_2022 = row['company_2018_2022']
    company_2013_2017 = row['company_2013_2017']

  
    rank_2013_2017 = ranking_2013_2017_dc.get(company_2018_2022, None)
    rank_2018_2022 = ranking_2018_2022_dc.get(company_2018_2022, None)

    if rank_2013_2017 is None:
        return '+'
    elif rank_2013_2017 == rank_2018_2022:
        return '='
    elif rank_2013_2017 > rank_2018_2022:
        return '+'
    else:
        return '-'

merged_top_25_dc['ranking_change'] = merged_top_25_dc.apply(ranking_change, axis=1)

merged_top_25_dc

#### Observations from the Results

From the resulting table, we can observe several key trends in the **Digital Communication** field:

- **QUALCOMM (US)** retains its position as the leading company in both periods, with a slight increase in its market share from 19.6% in 2013-2017 to 20.0% in 2018-2022, indicating steady innovation in digital communication technologies.
- **LG Electronics (KR)**, ranked second in 2013-2017, drops to fourth place in 2018-2022. Despite this, its share remains strong at 13.3% in the earlier period and 9.7% in the latter, showing consistent activity.
- **SAMSUNG ELECTRONICS (KR)** maintains a significant presence, though it drops from third to fourth place, with a slight decrease in share from 12.3% to 8.8%.
- **HUAWEI TECHNOLOGIES (CN)**, previously ranked fifth in 2013-2017, climbs to second place in 2018-2022, reflecting an increase in its share from 7.3% to 13.6%.
- **ZTE CORPORATION (CN)** sees an increase in its rank and share, moving up from 8th to 6th place, with a rise in share from 2.3% to 3.1%.
- **BYTEDANCE (US)**, a relatively newer player, enters the top 10 in 2018-2022, marking a notable rise in innovation in the digital communication space.

The ranking changes also indicate the increasing influence of companies from China, such as **Huawei** and **ZTE**, which have gained ground over the period. Established players like **Sony**, **Intel**, and **Panasonic** have seen minor shifts, but they remain central to the digital communication landscape.

This analysis highlights the growing presence of Chinese companies, as well as the continued strength of key players from the US, Japan, and Europe, showcasing the competitive and dynamic nature of the digital communication sector.





